# Introdution

In [1]:
%%capture
%load_ext sql
%sql sqlite:///factbook.db

# Overview of the Data

We'll begin by getting a sense of what the data looks like.

In [2]:
%%sql
SELECT *
  FROM facts
LIMIT 5;

 * sqlite:///factbook.db
Done.


id,code,name,area,area_land,area_water,population,population_growth,birth_rate,death_rate,migration_rate
1,af,Afghanistan,652230,652230,0,32564342,2.32,38.57,13.89,1.51
2,al,Albania,28748,27398,1350,3029278,0.3,12.92,6.58,3.3
3,ag,Algeria,2381741,2381741,0,39542166,1.84,23.67,4.31,0.92
4,an,Andorra,468,468,0,85580,0.12,8.13,6.96,0.0
5,ao,Angola,1246700,1246700,0,19625353,2.78,38.78,11.49,0.46


Here are the descriptions for some of the columns:

* `name` - The name of the country
* `area` - The total land and sea area of the country
* `population` - The country's population
* `population_growth` - The country's population growth as a percentage
* `birth_rate` - The country's birth rate, or the number of births a year per 1,000 people
* `death_rate` - The country's death rate, or the number of death a year per 1,000 people
* `area` - The country's total area (both land and water)
* `area_land` - The country's land area in square kilometers
* `area_water` - The country's waterarea in square kilometers



# Summary Statistics

In [3]:
%%sql
SELECT MIN(population) AS min_pop, 
       MAX(population) AS max_pop, 
       MIN(population_growth) AS min_pop_growth, 
       MAX(population_growth) AS max_pop_growth
    FROM facts;

 * sqlite:///factbook.db
Done.


min_pop,max_pop,min_pop_growth,max_pop_growth
0,7256490011,0.0,4.02


From the results obtained it is as if there is a country in which all the people of the world live (7.2 billion) and another in which no one lives. Let us therefore examine these two countries in detail.

In [4]:
%%sql
SELECT *
    FROM facts
    WHERE population == 0 OR population == (SELECT MAX(population) FROM facts);

 * sqlite:///factbook.db
Done.


id,code,name,area,area_land,area_water,population,population_growth,birth_rate,death_rate,migration_rate
250,ay,Antarctica,None,280000,None,0,None,None,None,None
261,xx,World,None,None,None,7256490011,1.08,18.6,7.8,None


Therefore, the World and the continent of Antarctica are considered countries in the database.

Now that we know this, we should recalculate the summary statistics we calculated earlier, while excluding the row for the whole world.

# Summary Statistics Revisited

In [5]:
%%sql
SELECT MIN(population) AS min_pop, 
       MAX(population) AS max_pop, 
       MIN(population_growth) AS min_pop_growth, 
       MAX(population_growth) AS max_pop_growth
    FROM facts
    WHERE name != 'World';

 * sqlite:///factbook.db
Done.


min_pop,max_pop,min_pop_growth,max_pop_growth
0,1367485388,0.0,4.02


In [6]:
%%sql
SELECT name, population
    FROM facts
    WHERE population = (SELECT MAX(population) 
                            FROM facts
                            WHERE name != 'World');

 * sqlite:///factbook.db
Done.


name,population
China,1367485388


There's a country whose population closes in on 1.4 billion and it is China.

## Which countries are most populated?

In [7]:
%%sql
 SELECT name, population
    FROM facts
    WHERE name != 'World'
    ORDER BY population DESC
    LIMIT 5;

 * sqlite:///factbook.db
Done.


name,population
China,1367485388
India,1251695584
European Union,513949445
United States,321368864
Indonesia,255993674


We can see that only China and India have a population greater than 1 billion.

## Which Countries are showing stronger growth? 

In [8]:
%%sql
 SELECT name, population_growth
    FROM facts
    WHERE name != 'World'
    ORDER BY population_growth DESC
    LIMIT 5;

 * sqlite:///factbook.db
Done.


name,population_growth
South Sudan,4.02
Malawi,3.32
Burundi,3.28
Niger,3.25
Uganda,3.24


Poorer countries, particularly Afrian Countries, show a stronger growth. 

## Exploring Average Population and Area

In [9]:
%%sql
SELECT AVG(population) AS avg_pop, 
       AVG(area) AS avg_area
    FROM facts
    WHERE name != 'World';

 * sqlite:///factbook.db
Done.


avg_pop,avg_area
32242666.56846473,555093.546184739


We can see that the average population is 32 million, while the average total area of a country is 555 thousand square kilometers.

## Finding Densely Populated Countries

In conclusion, we visualize the most populated countries, identifying them as those that:
- have a population above the world average
- have a surface area smaller than the world average

In [10]:
%%sql
SELECT *
    FROM facts
    WHERE population > (SELECT AVG(population)
                        FROM facts
                        WHERE name != 'World'
                       ) AND
          area < (SELECT AVG(area)
                FROM facts
                WHERE name != 'World'
                )
    ORDER BY population, area;

 * sqlite:///factbook.db
Done.


id,code,name,area,area_land,area_water,population,population_growth,birth_rate,death_rate,migration_rate
120,mo,Morocco,446550,446300,250,33322699,1.0,18.2,4.81,3.36
80,iz,Iraq,438317,437367,950,37056169,2.93,31.45,3.77,1.62
182,ug,Uganda,241038,197100,43938,37101745,3.24,43.79,10.69,0.74
139,pl,Poland,312685,304255,8430,38562189,0.09,9.74,10.19,0.46
163,sp,Spain,505370,498980,6390,48146134,0.89,9.64,9.04,8.31
91,ks,"Korea, South",99720,96920,2800,49115196,0.14,8.19,6.75,0.0
83,it,Italy,301340,294140,7200,61855120,0.27,8.74,10.19,4.1
185,uk,United Kingdom,243610,241930,1680,64088222,0.54,12.17,9.35,2.54
173,th,Thailand,513120,510890,2230,67976405,0.34,11.19,7.8,0.0
65,gm,Germany,357022,348672,8350,80854408,0.17,8.47,11.42,1.24


An alternative approach could be to consider the ratio between population and land area in each country as follows:

In [11]:
%%sql
SELECT name, ROUND(CAST(population AS FLOAT)/area, 2) AS pop_area
    FROM facts
    WHERE pop_area > (SELECT ROUND(CAST(population AS FLOAT)/area, 2) AS pop_area
                      FROM facts
                      WHERE name != 'World'
                     )
    ORDER BY pop_area DESC
    LIMIT 10;

 * sqlite:///factbook.db
Done.


name,pop_area
Macau,21168.96
Monaco,15267.5
Singapore,8141.28
Hong Kong,6445.04
Gaza Strip,5191.82
Gibraltar,4876.33
Bahrain,1771.86
Maldives,1319.64
Malta,1310.02
Bermuda,1299.93


With this methodology, Macau is the most densely populated country in the world.